### Questions

1. What are the strengths of K-means; when does it perform well?

    `K-means is super fast. it performs well when the clusters are fat and ball-shaped.`

2. What are the weaknesses of K-means; when does it perform poorly?

    `K-means cannot recognize bar-shaped clusters. It performs poorly when the shapes of clusters are complex.`

3. What makes K-means a good candidate for the clustering problem, if you have enough knowledge about the data?

    `K-means is a good choice when we only want a fast first-look into the data, or if the dataset is just simple and straight forward.`

### Import Libraries

In [7]:
import numpy as np
import cv2
import tqdm
import os
import sys
# color of different clusters
GBR = [[0, 0, 255],
       [0, 128, 255],
       [255, 0, 0],
       [128, 0, 128],
       [255, 0, 255]]

# path configuration
project_root = os.path.abspath('.')
output_path = os.path.join(project_root)
input_path = os.path.join(project_root)
if not os.path.exists(output_path):
    os.makedirs(output_path)

### Implement K-Means

In [8]:
def kmeans(data:np.ndarray, n_cl:int, seed:int=114, batch_size:int=1000):
    """
        K-means

    :param data:    original data
    :param n_cl:    number of classes
    :param seeds:   seeds
    :return:        new labels and new seeds
    """
    np.random.seed(seed)
    mini_data = data[np.random.choice(data.shape[0], batch_size, replace=False)]
    
    n_samples, n_channels = mini_data.shape

    # TODO: firstly you should init centroids by a certain strategy
    centers = mini_data[np.random.choice(n_samples, n_cl, replace=False)]

    old_labels = np.zeros((n_samples,), dtype=int)
    while True:
        # TODO: calc distance between samples and centroids
        distances = [[np.sum(np.square(mini_data[i] - centers[k])) for k in range(n_cl)] for i in range(n_samples)]

        # TODO: classify samples
        new_labels = np.zeros((n_samples,), dtype=int)
        for x in range(n_samples):
            new_labels[x] = np.argmin(distances[x])

        # TODO: update centroids
        distances = np.array(distances)
        if len(np.unique(new_labels)) < n_cl:
            print(np.unique(new_labels))
        for x in range(n_cl):
            centers[x] = np.mean(mini_data[np.argwhere(new_labels == x).flatten()])

        if np.all(new_labels == old_labels):
            break
        old_labels = new_labels

    n_samples, n_channels = data.shape
    print("last dist calc:")
    distances = [[np.sum(np.square(data[i] - centers[k])) for k in range(n_cl)] for i in tqdm.tqdm(range(n_samples))]
    new_labels = np.zeros((n_samples,), dtype=int)
    print("last label mark:")
    for x in tqdm.tqdm(range(n_samples)):
        new_labels[x] = np.argmin(distances[x])

    return new_labels

### Convert the Video

In [9]:

def detect(video, n_cl=2):
    # load video, get number of frames and get shape of frame
    cap = cv2.VideoCapture(video)
    fps = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    size = (int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)),
            int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)))

    # instantiate a video writer
    video_writer = cv2.VideoWriter(os.path.join(output_path, "result_with_%dclz.mp4" % n_cl),
                                   cv2.VideoWriter_fourcc(*'mp4v'),
                                   (fps / 10),
                                   size,
                                   isColor=True)

    # initialize frame and seeds
    ret, frame = cap.read()

    print("Begin clustering with %d classes:" % n_cl)
    # bar = tqdm.tqdm(total=fps)  # progress bar
    count = 0
    while ret:
        count += 1
        print(count, "/", fps)

        frame = np.float32(frame)
        h, w, c = frame.shape

        # k-means
        data = frame.reshape((h * w, c))
        labels = kmeans(data, n_cl=n_cl)

        # give different cluster different colors
        new_frame = np.zeros((h * w, c))
        # TODO: dye pixels with colors
        label_count = []
        for i in range(n_cl):
            label_count.append(np.count_nonzero(labels == i))
        color_idx = np.argsort(label_count)
        for x in range(h * w):
            new_frame[x] = GBR[color_idx[labels[x]]]
        new_frame = new_frame.reshape((h, w, c)).astype("uint8")
        video_writer.write(new_frame)

        ret, frame = cap.read()
        # bar.update()

    # release resources
    video_writer.release()
    cap.release()
    cv2.destroyAllWindows()


video_sample = os.path.join(input_path, "road_video.MOV")
detect(video_sample, n_cl=4)

Begin clustering with 4 classes:
1 / 35
last dist calc:


100%|██████████| 2073600/2073600 [00:52<00:00, 39781.15it/s]


last label mark:


100%|██████████| 2073600/2073600 [00:08<00:00, 230434.83it/s]


2 / 35
last dist calc:


...

100%|██████████| 2073600/2073600 [00:10<00:00, 206468.82it/s]


35 / 35
last dist calc:


100%|██████████| 2073600/2073600 [00:52<00:00, 39675.44it/s]


last label mark:


100%|██████████| 2073600/2073600 [00:09<00:00, 222618.49it/s]
